In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/santander-customer-transaction-prediction/sample_submission.csv
/kaggle/input/santander-customer-transaction-prediction/test.csv
/kaggle/input/santander-customer-transaction-prediction/train.csv


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import roc_auc_score
sns.set(style="darkgrid")

In [21]:
roc_auc_score([1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 1, 0])

0.6

In [3]:
sample = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/sample_submission.csv')
train = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')
test = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/test.csv')
train.drop('ID_code', axis=1, inplace=True)
test.drop('ID_code', axis=1, inplace=True)

In [ ]:
fig = plt.figure(figsize=(20,30))
for index,col in enumerate(train.iloc[:,1:40].columns):
    plt.subplot(10,4,index+1)
    sns.catplot(train[col], hue=train['target'])
fig.tight_layout(pad=1.0)

In [4]:
X = train.drop('target', axis=1)
y = train['target']

In [5]:
for df in [X, test]:
    #nc = ['sum', 'min', 'max', 'mean', 'std', 'skew', 'kurt', 'med']
    df['sum'] = df.iloc[:, :200].sum(axis=1)  
    df['min'] = df.iloc[:, :200].min(axis=1)
    df['max'] = df.iloc[:, :200].max(axis=1)
    df['mean'] = df.iloc[:, :200].mean(axis=1)
    df['std'] = df.iloc[:, :200].std(axis=1)
    df['skew'] = df.iloc[:, :200].skew(axis=1)
    df['kurt'] = df.iloc[:, :200].kurtosis(axis=1)
    df['med'] = df.iloc[:, :200].median(axis=1)

In [6]:
features = [c for c in X.columns if c not in ['ID_code', 'target']]
for feature in features:
    X['r2_'+feature] = np.round(X[feature], 2)
    test['r2_'+feature] = np.round(test[feature], 2)
    X['r1_'+feature] = np.round(X[feature], 1)
    test['r1_'+feature] = np.round(test[feature], 1)

In [ ]:
kfold = StratifiedKFold(n_splits=2)
random_state = 2
classifiers = []
#classifiers.append(SVC(random_state=random_state))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(LinearDiscriminantAnalysis())

In [14]:
param = {'num_leaves': 31, 'objective': 'binary', 'metric':'auc'}
from sklearn.model_selection import train_test_split
import lightgbm as lgb
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
num_round = 500
train_data = lgb.Dataset(X_train, label=y_train)
validation_data = lgb.Dataset(X_test, label=y_test)
fd = lgb.Dataset(X, label=y)
#cl = lgb.train(param, train_data, num_round, valid_sets=[validation_data], verbose_eval=100)
cl = lgb.train(param, fd, num_round)

In [15]:
roc_auc_score(y, cl.predict(X))

0.9832302061971468

In [45]:
folds = StratifiedKFold(n_splits=10, shuffle=False, random_state=44000)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])


    num_round = 1000000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


NameError: name 'train_df' is not defined

In [9]:
param = {
    'bagging_freq': 5,
    'bagging_fraction': 0.4,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.05,
    'learning_rate': 0.01,
    'max_depth': -1,  
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': 1
}

In [ ]:
cv_results = []
for classifier in classifiers :
    print(cross_val_score(classifier, X[:30000], y[:30000], scoring = "roc_auc", cv = kfold, n_jobs=4))
    print('done')
cv_means = []
cv_std = []
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

In [ ]:
for cl in classifiers:
    X_train, X_test, y_train, y_test = train_test_split(X[:30000], y[:30000], test_size=0.33, random_state=0)
    cl.fit(X_train, y_train)
    print(roc_auc_score(cl.predict(X_test), y_test))

In [ ]:
cl = LinearDiscriminantAnalysis()
cl.fit(X, y)

In [16]:
res = cl.predict(test)

In [17]:
sample['target'] = res
sample.to_csv('submission.csv', index=False)